In [2]:
"""
This example implements the experiments on citation networks from the paper:

Semi-Supervised Classification with Graph Convolutional Networks (https://arxiv.org/abs/1609.02907)
Thomas N. Kipf, Max Welling
"""
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess

# from spektral.layers.convolutional.gcn_conv import GCNConv

# Save the original call method
_original_gcn_conv_call = GCNConv.call

# Define a patched version that filters out a mask list containing Nones.
def _patched_gcn_conv_call(self, inputs, mask=None):
    # If the mask is a list where the first element is None, replace mask with None.
    if mask is not None and isinstance(mask, list) and mask[0] is None:
        mask = None
    return _original_gcn_conv_call(self, inputs, mask=mask)

# Apply the patch:
GCNConv.call = _patched_gcn_conv_call

learning_rate = 1e-2
seed = 0
epochs = 200
patience = 10
data = "cora"

tf.random.set_seed(seed=seed)  # make weight initialization reproducible

# Load data
dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])


# We convert the binary masks to sample weights so that we can compute the
# average loss over the nodes (following original implementation by
# Kipf & Welling)
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)


weights_tr, weights_va, weights_te = (
    mask_to_weights(mask)
    for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)

model = GCN(n_labels=dataset.n_labels)
model.compile(
    optimizer=Adam(learning_rate),
    loss=CategoricalCrossentropy(reduction="sum"),
    weighted_metrics=["acc"],
)

# Train model
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

# Evaluate model
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Pre-processing node features
Epoch 1/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/step - acc: 0.1857 - loss: 1.9533 - val_acc: 0.2740 - val_loss: 1.9497
Epoch 2/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.5286 - loss: 1.9456 - val_acc: 0.3360 - val_loss: 1.9447
Epoch 3/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - acc: 0.6429 - loss: 1.9368 - val_acc: 0.3300 - val_loss: 1.9394
Epoch 4/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - acc: 0.6214 - loss: 1.9272 - val_acc: 0.3120 - val_loss: 1.9345
Epoch 5/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.6143 - loss: 1.9175 - val_acc: 0.3160 - val_loss: 1.9299
Epoch 6/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.5929 - loss: 1.9076 - val_acc: 0.3220 - val_loss: 1.9255
Epoch 7/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.6143 - loss: 1.8975 - val_acc: 0.3260 - val_loss: 1.9211
Epoch 8/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - acc: 0.6500 - loss: 1.8873 - val_acc: 0.3360 - val_loss: 1.9169
Epoch 9/200
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

In [5]:
import tensorflow as tf
import tensorflow_probability as tfp
tfb = tfp.bijectors

N = 4                                  # matrix dimension
# 1. Unconstrained vector of parameters
raw = tf.Variable(tf.random.normal([N * (N + 1) // 2]))*1

# 2. Bijector packs the vector into a lower-triangular L
#    and applies softplus to the diagonal so it stays positive.
fill_tril = tfb.FillScaleTriL(diag_bijector=tfb.Softplus(), diag_shift=1e-5)  # ε-shift avoids zeros
L = fill_tril(raw)                       # shape (N, N)

# 3. SPD covariance (or kernel, or weight matrix…)
Sigma = tf.matmul(L, L, transpose_b=True)

print(Sigma)

tf.Tensor(
[[ 0.5718629  -0.459951   -0.4157904   0.64199835]
 [-0.459951    1.8181738   1.0801171  -1.5339191 ]
 [-0.4157904   1.0801171   3.1469738  -1.1656852 ]
 [ 0.64199835 -1.5339191  -1.1656852   1.4713585 ]], shape=(4, 4), dtype=float32)
